# Homework 3

Working with Discrete Event Simulation in Python using SimPy.

## Preliminaries

In [1]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import optimize
from numpy.random import default_rng
from IPython.display import Image
import simpy
import my_functions

In [2]:
%matplotlib inline

## Work

The process I am going to use for my Discrete Event Process is a Blood Donation process.

```
Arrival --> Registration --> Iron Level Check --> Blood Collection --> Schedule Next Donation (if desired) --> Wait 15 minutes --> Exit
```


In [6]:
def patient_arrivals(env, interarrival_time=5.0):
    """Generate patients according to a fixed time arrival process"""

    # Create a counter to keep track of number of patients generated and to serve as unique patient id
    patient = 0

    # Infinite loop for generating patients
    while True:

        # Generate next interarrival time (this will be more complicated later)
        iat = interarrival_time
        
        # This process will now yield to a 'timeout' event. This process will resume after iat time units.
        yield env.timeout(iat)

        # Okay, we're back. :) New patient generated = update counter of patients
        patient += 1
        
        print(f"Patient {patient} created at time {env.now}")

In [7]:
# Initialize a simulation environment
env1 = simpy.Environment()

# Create a process generator and start it and add it to the env
# env.process() starts the patient_arrivals process and adds it to the environment
runtime = 25
interarrival_time = 3.0
env1.process(patient_arrivals(env1, interarrival_time))

# Run the simulation
env1.run(until=runtime)

Patient 1 created at time 3.0
Patient 2 created at time 6.0
Patient 3 created at time 9.0
Patient 4 created at time 12.0
Patient 5 created at time 15.0
Patient 6 created at time 18.0
Patient 7 created at time 21.0
Patient 8 created at time 24.0


In [8]:
def simplified_blood_donation(env, name, mean_prebd_time, mean_bd_time, mean_postbd_time, bd_tech):
    """Process function modeling how a patient flows through system."""

    print(f"{name} entering blood donation clinic at {env.now:.4f}")
    
    # Yield for the pre-blood donation time
    yield env.timeout(rg.exponential(mean_prebd_time))
    
    # Request blood draw technician to start blood draw
    with bd_tech.request() as request:
        print(f"{name} requested blood draw tech at {env.now:.4f}")
        yield request
        print(f"{name} got blood draw tech at {env.now:.4f}")
        yield env.timeout(rg.normal(mean_bd_time, 0.5))
                          
    # Yield for the post-blood donation time
    yield env.timeout(mean_postbd_time)
    
    # The process is over, we would exit the clinic
    print(f"{name} exiting blood donation clinic at {env.now:.4f}")

In [8]:
rg = default_rng(seed=4470)

In [8]:
def patient_arrivals_random_2(env, mean_interarrival_time, mean_prebd_time, mean_bd_time,
                              mean_postbd_time, bd_tech, rg=default_rng(0)):
    """Generate patients according to a Poisson arrival process"""

    # Create a counter to keep track of number of patients generated and to serve as unique patient id
    patient = 0

    # Infinite loop for generating patients
    while True:

        # Generate next interarrival time
        iat = rg.exponential(mean_interarrival_time)
        
        # This process will now yield to a 'timeout' event. This process will resume after iat time units.
        yield env.timeout(iat)

        # Update counter of patients
        patient += 1

        print(f"Patient{patient} created at time {env.now}")
               
        # Create and register the simplifed blood donation process in two steps
        
        # Create a new patient delay process generator object.
        patient_visit = simplified_blood_donation(env, 'Patient{}'.format(patient), 
                                               mean_prebd_time, mean_bd_time, mean_postbd_time, bd_tech)
        
        # Register the process with the simulation environment
        env.process(patient_visit)
        
        # Here's the one step version
        # env.process(simplified_blood_donation(env, 'Patient{}'.format(patient), 
        #                                      mean_prebd_time, mean_bd_time, mean_postbd_time, bd_tech))

In [9]:
# Initialize a simulation environment
env3 = simpy.Environment()

# Set input values
mean_interarrival_time = 3.0
mean_prebd_time = 4.25
mean_bd_time = 4.0
mean_postbd_time = 15.0
num_bd_tech = 2

# Create vaccinator resource
bd_tech = simpy.Resource(env3, num_bd_tech)

# Register our new arrivals process
env3.process(patient_arrivals_random_2(env3, mean_interarrival_time, mean_prebd_time, mean_bd_time,
                                      mean_postbd_time, bd_tech))

# Run the simulation
runtime = 50
env3.run(until=runtime)

Patient1 created at time 2.039795711906729
Patient1 entering blood donation clinic at 2.0398
Patient2 created at time 5.098587016304323
Patient2 entering blood donation clinic at 5.0986
Patient3 created at time 5.158007004071489
Patient3 entering blood donation clinic at 5.1580
Patient4 created at time 5.164814984115174
Patient4 entering blood donation clinic at 5.1648
Patient1 requested blood draw tech at 5.3441
Patient1 got blood draw tech at 5.3441
Patient2 requested blood draw tech at 5.4710
Patient2 got blood draw tech at 5.4710
Patient4 requested blood draw tech at 6.7192
Patient5 created at time 6.815843602032318
Patient5 entering blood donation clinic at 6.8158
Patient4 got blood draw tech at 9.3543
Patient3 requested blood draw tech at 10.0950
Patient3 got blood draw tech at 10.0950
Patient6 created at time 11.705664906007474
Patient6 entering blood donation clinic at 11.7057
Patient6 requested blood draw tech at 12.4201
Patient5 requested blood draw tech at 13.0868
Patient7 c